In [2]:
import pandas as pd
import numpy as np

In [3]:
qwen_7B_init = pd.read_csv('inference_results_qwen_7B_init.csv')
qwen_7B_sft = pd.read_csv('inference_results_qwen_7B_sft.csv')
qwen_14B_init = pd.read_csv('inference_results_qwen_14B_init.csv')
qwen_14B_sft = pd.read_csv('inference_results_qwen_14B_sft.csv')


In [37]:
# 比较正确率,召回率,查准率, F1值
acc_set = []
recall_set = []
precision_set = []
f1_set = []

data_set = [qwen_7B_init, qwen_7B_sft, qwen_14B_init, qwen_14B_sft]
res_set = []
for data in data_set:
    res = []
    for i in range(300):
        label = data['Expected Output'][i]
        pred = data['Generated Output'][i]
        res.append([label, pred])
    res_set.append(res)
    

# 计算正确率
for i in range(4):
    acc = 0
    for j in range(300):
        if res_set[i][j][0] == res_set[i][j][1]:
            acc += 1
    acc_set.append(acc / 300)

# # 计算召回率
# for i in range(4):
#     tp = 0
#     fn = 0
#     for j in range(300):
#         if res_set[i][j][0] == res_set[i][j][1] and res_set[i][j][0] == 1:
#             tp += 1
#         if res_set[i][j][0] == 1:
#             fn += 1
#     if tp + fn == 0:
#         recall_set.append('-')
#     else:
#         recall_set.append(tp / (tp + fn))

# # 计算查准率
# for i in range(4):
#     tp = 0
#     fp = 0
#     for j in range(300):
#         if res_set[i][j][0] == res_set[i][j][1] and res_set[i][j][0] == 1:
#             tp += 1
#         if res_set[i][j][1] == 1:
#             fp += 1
#     if tp + fp == 0:
#         precision_set.append('-')
#     else:
#         precision_set.append(tp / (tp + fp))

# # 计算F1值
# for i in range(4):
#     if precision_set[i] == '-' or recall_set[i] == '-':
#         f1_set.append('-')
#     else:
#         f1 = 2 * precision_set[i] * recall_set[i] / (precision_set[i] + recall_set[i])
#         if precision_set[i] == '-' or recall_set[i] == '-':
#             f1_set.append('-')
#         else:
#             f1_set.append(f1)
print('7B_init:', acc_set[0])
print('7B_sft:', acc_set[1])
print('14B_init:', acc_set[2])
print('14B_sft:', acc_set[3])
# print(recall_set)
# print(precision_set)
# print(f1_set)


7B_init 0.25
7B_sft 0.5433333333333333
14B_init 0.38
14B_sft 0.49


In [10]:
from sklearn.metrics import precision_score, recall_score, f1_score

data_set = [qwen_7B_init, qwen_7B_sft, qwen_14B_init, qwen_14B_sft]
res_set = []

# 存储结果：每个数据集的标签和预测
for data in data_set:
    res = []
    for i in range(300):
        label = data['Expected Output'][i]
        pred = data['Generated Output'][i]
        res.append([label, pred])
    res_set.append(res)

# 计算正确率、Precision、Recall 和 F1 score
acc_set = []
precision_set = []
recall_set = []
f1_set = []

for i in range(4):
    labels = [res_set[i][j][0] for j in range(300)]
    preds = [res_set[i][j][1] for j in range(300)]
    
    # 计算正确率
    acc = sum(1 for j in range(300) if labels[j] == preds[j]) / 300
    acc_set.append(acc)
    
    # 计算 Precision, Recall, F1 Score（macro 平均）
    precision = precision_score(labels, preds, average='weighted', zero_division=0)
    recall = recall_score(labels, preds, average='weighted', zero_division=0)
    f1 = f1_score(labels, preds, average='weighted', zero_division=0)
    
    precision_set.append(precision)
    recall_set.append(recall)
    f1_set.append(f1)

# 输出结果
for i, model in enumerate(['qwen_7B_init', 'qwen_7B_sft', 'qwen_14B_init', 'qwen_14B_sft']):
    print(f"Results for {model}:")
    print(f"Accuracy: {acc_set[i]:.4f}")
    print(f"Precision: {precision_set[i]:.4f}")
    print(f"Recall: {recall_set[i]:.4f}")
    print(f"F1 Score: {f1_set[i]:.4f}")
    print("-" * 30)


Results for qwen_7B_init:
Accuracy: 0.2500
Precision: 0.4802
Recall: 0.2500
F1 Score: 0.2814
------------------------------
Results for qwen_7B_sft:
Accuracy: 0.5433
Precision: 0.4963
Recall: 0.5433
F1 Score: 0.5186
------------------------------
Results for qwen_14B_init:
Accuracy: 0.3800
Precision: 0.4226
Recall: 0.3800
F1 Score: 0.3952
------------------------------
Results for qwen_14B_sft:
Accuracy: 0.4900
Precision: 0.4502
Recall: 0.4900
F1 Score: 0.4693
------------------------------


In [9]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

data_set = [qwen_7B_init, qwen_7B_sft, qwen_14B_init, qwen_14B_sft]
model_names = ['qwen_7B_init', 'qwen_7B_sft', 'qwen_14B_init', 'qwen_14B_sft']

# 结果列表
res_set = []

# 提取标签和预测
for data in data_set:
    res = []
    for i in range(300):
        label = data['Expected Output'][i]
        pred = data['Generated Output'][i]
        res.append([label, pred])
    res_set.append(res)

# 计算每个类别的指标
for i, model in enumerate(model_names):
    print(f"Results for {model}:")
    
    labels = [res_set[i][j][0] for j in range(300)]
    preds = [res_set[i][j][1] for j in range(300)]
    
    # 获取所有唯一类别
    classes = np.unique(labels)
    
    # 为每个类别计算指标
    for cls in classes:
        cls_labels = [1 if label == cls else 0 for label in labels]
        cls_preds = [1 if pred == cls else 0 for pred in preds]

        
        acc = accuracy_score(cls_labels, cls_preds)
        precision = precision_score(cls_labels, cls_preds, average='weighted', zero_division=0)
        recall = recall_score(cls_labels, cls_preds, average='weighted', zero_division=0)
        f1 = f1_score(cls_labels, cls_preds, average='weighted', zero_division=0)
        
        print(f"  Class: {cls}")
        print(f"    Accuracy: {acc:.4f}")
        print(f"    Precision: {precision:.4f}")
        print(f"    Recall: {recall:.4f}")
        print(f"    F1 Score: {f1:.4f}")
    
    print("-" * 40)


Results for qwen_7B_init:
  Class: high
    Accuracy: 0.4500
    Precision: 0.6222
    Recall: 0.1591
    F1 Score: 0.2534
  Class: low
    Accuracy: 0.9567
    Precision: 0.0000
    Recall: 0.0000
    F1 Score: 0.0000
  Class: medium
    Accuracy: 0.5267
    Precision: 0.3456
    Recall: 0.4700
    F1 Score: 0.3983
  Class: very high
    Accuracy: 0.8200
    Precision: 0.0000
    Recall: 0.0000
    F1 Score: 0.0000
----------------------------------------
Results for qwen_7B_sft:
  Class: high
    Accuracy: 0.5633
    Precision: 0.6142
    Recall: 0.6875
    F1 Score: 0.6488
  Class: low
    Accuracy: 0.9667
    Precision: 0.0000
    Recall: 0.0000
    F1 Score: 0.0000
  Class: medium
    Accuracy: 0.6033
    Precision: 0.4078
    Recall: 0.4200
    F1 Score: 0.4138
  Class: very high
    Accuracy: 0.9533
    Precision: 0.0000
    Recall: 0.0000
    F1 Score: 0.0000
----------------------------------------
Results for qwen_14B_init:
  Class: high
    Accuracy: 0.5233
    Precision: 0.

In [6]:
np.unique(labels)

array(['high', 'low', 'medium', 'very high'], dtype='<U9')